1. Read in the boundaries for each admin
2. Create a task for each boundary for gee to:
    
    2.1 Get embeddings product
    
    2.2 Aggregate pixels for each boundary
    
    2.3 Save to google drive 

In [39]:
import ee
import yaml
import pandas as pd
import geopandas as gpd
import leafmap.leafmap as leafmap
from shapely.geometry import mapping

In [2]:
# step  1: Read the shapefile for the boundaries
boundary_path = r"C:/Users/Sean O Heir/Documents/research/data/mozambique/boundaries/moz_admbnda_adm3_ine_20190607.shp"
admin_boundary_gdf = gpd.read_file(boundary_path)

In [6]:
def establish_connection_with_earth_engine(project_name):
    """
    Authenticates and initializes a connection with Google Earth Engine.

    Args:
        project_name (str): The name of the Google Cloud project to use for authentication.
    """
    try:
        ee.Initialize()
    except Exception as e:
        ee.Authenticate()
        ee.Initialize(project=project_name)
        print(ee.String('Connection with the Earth Engine servers established').getInfo())

establish_connection_with_earth_engine("ee-soheir-dev")

Connection with the Earth Engine servers established


In [16]:
# function I may need to convert polygon object to ee polygon
def convert_shapefile_to_ee_geometry(shp_geometry):
    """
    Converts a polygon boundary to an Earth Engine geometry object.

    Returns:
        ee.Geometry: An Earth Engine Geometry object representing the shapefile boundary.
    """
    # Convert the geometry to GeoJSON format
    boundary_geojson = mapping(shp_geometry)

    # Create and return an Earth Engine Geometry from the GeoJSON
    return ee.Geometry(boundary_geojson)

In [20]:
# step 2: Create a task for each boundary in GEE
test_boundary = admin_boundary_gdf.geometry[10]
ee_boundary = convert_shapefile_to_ee_geometry(test_boundary)

In [ ]:
dataset = ee.ImageCollection('GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL')
dataset

In [35]:
embeddings = dataset.filterDate('2023-01-01', '2024-01-01').filterBounds(ee_boundary).first()

In [61]:
# Compute the mean of each band over the ee_boundary
mean_dict = embeddings.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=ee_boundary,
    # scale=30,  # Adjust this depending on your data resolution
    maxPixels=1e13
)

In [62]:
# Get the result as a Python dictionary
mean_values = mean_dict.getInfo()

pd.DataFrame([mean_values])

,A00,A01,A02,A03,A04,A05,A06,A07,A08,A09,...,A54,A55,A56,A57,A58,A59,A60,A61,A62,A63
0,0.018023,0.110296,-0.050106,0.007394,-0.131919,0.078577,0.092774,0.306607,-0.106542,0.076257,...,-0.108592,-0.155168,0.022579,-0.058929,-0.013118,0.039696,0.083952,-0.062258,0.046578,-0.189419


In [67]:
dataset = ee.ImageCollection('GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL')
results = []

# Loop through rows in the GeoDataFrame (geometry + ADM3_PCODE)
for _, row in admin_boundary_gdf[['ADM3_PCODE', 'geometry']][0:3].iterrows():
    adm3_code = row['ADM3_PCODE']
    geom = row['geometry']

    # Convert shapely geometry to ee.Geometry
    ee_geom = convert_shapefile_to_ee_geometry(geom)

    # Filter and get the first image
    embeddings = dataset.filterDate('2023-01-01', '2024-01-01').filterBounds(ee_geom).first()
    if embeddings is None:
        continue

    # Reduce to mean values
    mean_dict = embeddings.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=ee_geom,
        scale=10,       # Adjust as needed
        maxPixels=1e13
    )

    try:
        mean_values = mean_dict.getInfo()
        mean_values['ADM3_PCODE'] = adm3_code  
        results.append(mean_values)
    except Exception as e:
        print(f"Failed for {adm3_code}: {e}")
        continue

# Convert to DataFrame
embeddings_df = pd.DataFrame(results)

Send task to server-side

In [71]:
import math

# Load dataset and filter it once
dataset = ee.ImageCollection('GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL') \
    .filterDate('2023-01-01', '2024-01-01')

# Choose batch size
batch_size = 20
total_regions = len(admin_boundary_gdf)

for i in range(0, total_regions, batch_size):
    print(f"Processing batch {i} to {min(i+batch_size, total_regions)}...")

    # Slice the GeoDataFrame for this batch
    gdf_chunk = admin_boundary_gdf.iloc[i:i + batch_size]

    features = []

    for _, row in gdf_chunk[['ADM3_PCODE', 'geometry']].iterrows():
        adm3_code = row['ADM3_PCODE']
        geom = row['geometry']
        ee_geom = convert_shapefile_to_ee_geometry(geom)

        image = dataset.filterBounds(ee_geom).first()
        if image:
            mean_dict = image.reduceRegion(
                reducer=ee.Reducer.mean(),
                geometry=ee_geom,
                scale=10,
                maxPixels=1e13
            )

            feature = ee.Feature(None, mean_dict).set('ADM3_PCODE', adm3_code)
            features.append(feature)

    # Only export if there are features
    if features:
        feature_collection = ee.FeatureCollection(features)

        task = ee.batch.Export.table.toDrive(
            collection=feature_collection,
            description=f'satellite_embeddings_batch_{i}',
            folder='EarthEngineExports',
            fileNamePrefix=f'adm3_embeddings_2023_batch_{i}',
            fileFormat='CSV'
        )
        task.start()
        print(f"Export task for batch {i} started.")
    else:
        print(f"No valid features in batch {i}, skipping export.")

Processing batch 0 to 20...
Export task for batch 0 started.
Processing batch 20 to 40...
Export task for batch 20 started.
Processing batch 40 to 60...
Export task for batch 40 started.
Processing batch 60 to 80...
Export task for batch 60 started.
Processing batch 80 to 100...
Export task for batch 80 started.
Processing batch 100 to 120...
Export task for batch 100 started.
Processing batch 120 to 140...
Export task for batch 120 started.
Processing batch 140 to 160...
Export task for batch 140 started.
Processing batch 160 to 180...
Export task for batch 160 started.
Processing batch 180 to 200...
Export task for batch 180 started.
Processing batch 200 to 220...
Export task for batch 200 started.
Processing batch 220 to 240...
Export task for batch 220 started.
Processing batch 240 to 260...
Export task for batch 240 started.
Processing batch 260 to 280...
Export task for batch 260 started.
Processing batch 280 to 300...
Export task for batch 280 started.
Processing batch 300 to 32

### Processing the embeddings
After downloading from google drive, collect them into a single table and label them with population

In [72]:
import os 

In [82]:
path_to_embeddings = r"C:\Users\Sean O Heir\Documents\research\data\mozambique\embeddings"

# Collect only CSV files
fpaths = [os.path.join(path_to_embeddings, file) for file in os.listdir(path_to_embeddings) if file.endswith('.csv')]

# Read and concatenate all CSVs
embeddings_df = pd.concat([pd.read_csv(f) for f in fpaths], ignore_index=True)
embeddings_df = embeddings_df.drop(columns=['system:index', '.geo'])

In [98]:
adm3_features = r"C:\Users\Sean O Heir\Documents\research\data\mozambique\extracted_features\MOZ_population_adm3_features_2.csv"

adm3_features = pd.read_csv(adm3_features)
pop_df = adm3_features[['ADM3_PCODE', 'T_TL', 'admin_boundary_area']]
pop_df['pop_density'] = pop_df['T_TL'] / pop_df['admin_boundary_area']
pop_df = pop_df.drop(columns=['T_TL', 'admin_boundary_area'])

In [99]:
# merge embeddings with pop df 
features_pop_df = pd.merge(embeddings_df, pop_df, on="ADM3_PCODE")

output_path = r"C:\Users\Sean O Heir\Documents\research\data\mozambique\embeddings_and_labels.csv"

features_pop_df.to_csv(output_path, index=False) 